In [1]:
#!pip install jieba
import pandas as pd
import jieba
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd
import io
import os
import re
from collections import Counter

In [2]:
def CJK_cleaner(string): 
    filters = re.compile(u'[^0-9a-zA-Z\u4e00-\u9fff]+', re.UNICODE)
    return filters.sub('', string) 

## Part0: modify the path in your environment and k-mean paramemters 

In [23]:
file = '00_DataBase/analysis_keywordlist_nodatafill.csv'
categoryfile = '00_DataBase/Categorylist.txt'

cluster_num = 10
topN = 8

df = pd.read_csv(file)
dcat = pd.read_csv(categoryfile,sep='\s+',names=['name'])

## Part1: No consider how many products comsumers buy


In [18]:
for cc in range(2,4):#len(dcat)):
    cate_name =  dcat.values[cc][0]
    print(cate_name)
    df_cat = df[df['category']==cate_name]
    df_cat = pd.concat([df_cat['item_name'], df_cat['item']],axis=1)
    
    try:
        productlist = [] 
        for i in range(len(df_cat)):
            name = df_cat['item_name'].values[i]
            name = CJK_cleaner(name)
            name_cut = ' '.join(jieba.cut(name))
            productlist.append(name_cut)

        # print(productlist[5:10])

        df_cat_space = pd.DataFrame({'item_name': productlist,
                              'item': df_cat['item']})

        # word transform to array type
        c2vec = CountVectorizer()
        X = c2vec.fit_transform(df_cat_space['item_name'])
        X_cosine = cosine_similarity(X)


        kmeans = KMeans(n_clusters = cluster_num, init = 'k-means++', random_state = 42)
        y_kmeans = kmeans.fit_predict(X_cosine)

        df_topN=pd.DataFrame()
        for j in range(cluster_num):
            tlst=[]
            temp_y_kmeans = y_kmeans == j
            name_cluster = df_cat_space['item_name'][temp_y_kmeans]
            num_cluster = df_cat_space['item'][temp_y_kmeans]
            #========================================the results of clustering 
            if '/' in cate_name :
                folder_cate_name = cate_name.replace('/','_')
            else:
                folder_cate_name = cate_name
            if not os.path.exists(f'NoRepeat_{cluster_num}-{topN}/{folder_cate_name}'):
                os.makedirs(f'NoRepeat_{cluster_num}-{topN}/{folder_cate_name}')
            df_cluster = pd.concat([name_cluster,num_cluster],axis=1)
            df_cluster.to_csv(f'NoRepeat_{cluster_num}-{topN}/{folder_cate_name}/Detailed_Cluster_group{j}.csv',index=False)

            #========================================calculate the number
            #clear 
            name_cluster_list = name_cluster.tolist()
            for name in name_cluster_list:
                name = name.replace('\'', '').strip()
                name_array = name.split(' ')
                for nameset in name_array:
                    tlst.append(nameset)    
            #count
            name_counter = Counter(tlst)
            name_counter_topN = Counter(name_counter).most_common(topN)
            #fint topN 
            keywordlist = []
            numberlist  = []
            # print(name_counter_topN)
            for name_rank in name_counter_topN:
                keyword = name_rank[0]
                number  = str(name_rank[1])

                keywordlist.append(keyword)
                numberlist.append(number)

            if len(name_counter_topN) < topN:
                for nn in range(topN- len(name_counter_topN)):
                    keywordlist.append(np.nan) 
                    numberlist.append(np.nan)
            else:
                pass

            temp_df = pd.DataFrame({
                f'clusterName_{j}': keywordlist,
                f'clusterNum_{j}': numberlist,
            })

            df_topN = pd.concat([df_topN,temp_df],axis=1)

        df_topN.to_csv(f'NoRepeat_{cluster_num}-{topN}/{folder_cate_name}/{folder_cate_name}_Cluster_{topN}.csv',index=False)
    except:
        print('eeee')
        if not os.path.exists(f'nodata.log'):
             !echo "`date +%Y%m%dT%H%M%S`" > nodata.log
             cmd ='''
             echo %(cate_name)s no data >> nodata.log
             '''%locals()
        else:
             cmd ='''
             echo %(cate_name)s no data >> nodata.log
             '''%locals()                
        os.system(cmd)

髮部清潔/保養
身體清潔/保養


/Users/tong/opt/anaconda3/envs/scikit_xgb/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/Users/tong/opt/anaconda3/envs/scikit_xgb/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [47]:
df_topN

,clusterName_0,clusterNum_0,clusterName_1,clusterNum_1,clusterName_2,clusterNum_2,clusterName_3,clusterNum_3,clusterName_4,clusterNum_4,clusterName_5,clusterNum_5,clusterName_6,clusterNum_6,clusterName_7,clusterNum_7,clusterName_8,clusterNum_8,clusterName_9,clusterNum_9
0,抑菌,12,片,10,熟睡,19,蕾,5,靠得住,14,蘇菲,6,超薄,13,康乃馨,11,衛生,15,草本,9
1,衛生,12,棉,8,棉,12,妮,5,片,8,清新,5,片,6,片,5,棉,10,抑菌,6
2,Mdmmd,11,入,7,片,12,亞極,5,茶樹,6,片,5,日用,6,護墊,4,褲型,6,天然,3
3,新涼感,9,蕾,6,細,10,吸好眠,5,舒涼,6,涼感,4,蘇菲彈力,5,墊,3,片,6,潔翼,3
4,棉,5,好,6,緻,10,衛生,5,棉,6,微涼,2,潔翼,4,香,2,靠得住,5,片,3
5,片,4,自,6,柔,10,棉,5,用,4,小黃瓜,2,護墊,4,多,2,熟睡,3,溫柔,3
6,超涼感,3,在,6,蘇菲超,8,晚安,4,棉夜,3,衛生,2,貼身,3,棉,2,內,3,宣言,3
7,28cm,3,妮,5,超薄,8,褲褲,4,棉衛生,3,棉,2,型,2,一般,2,導,3,蘇菲極淨,2


## Part2: Consider how many products comsumers buy (header = "item")


In [24]:
for cc in [2,3,5,25,25,27,28,29,30,40,41]:#range(len(dcat)):
    cate_name =  dcat.values[cc][0]
    # print(cate_name)
    df_cat = df[df['category']==cate_name]
    df_cat = pd.concat([df_cat['item_name'], df_cat['item']],axis=1)
    try:
        productlist = [] 
        for i in range(len(df_cat)):
            name = df_cat['item_name'].values[i]
            name = CJK_cleaner(name)
            name_cut = ' '.join(jieba.cut(name))
            productlist.append(name_cut)

        # print(productlist[5:10])

        df_cat_space = pd.DataFrame({'item_name': productlist,
                              'item': df_cat['item']})

        # word transform to array type
        c2vec = CountVectorizer()
        X = c2vec.fit_transform(df_cat_space['item_name'])
        X_cosine = cosine_similarity(X)


        kmeans = KMeans(n_clusters = cluster_num, init = 'k-means++', random_state = 42)
        y_kmeans = kmeans.fit_predict(X_cosine)

        df_topN=pd.DataFrame()
        for j in range(cluster_num):
            tlst=[]
            temp_y_kmeans = y_kmeans == j
            name_cluster = df_cat_space['item_name'][temp_y_kmeans]
            num_cluster = df_cat_space['item'][temp_y_kmeans]
            
            #========================================the results of clustering 
            if '/' in cate_name :
                folder_cate_name = cate_name.replace('/','_')
            else:
                folder_cate_name = cate_name
            if not os.path.exists(f'Repeat_{cluster_num}-{topN}/{folder_cate_name}'):
                os.makedirs(f'Repeat_{cluster_num}-{topN}/{folder_cate_name}')
            df_cluster = pd.concat([name_cluster,num_cluster],axis=1)
            df_cluster.to_csv(f'Repeat_{cluster_num}-{topN}/{folder_cate_name}/Detailed_Cluster_group{j}.csv',index=False)
            #========================================calculate the number
            #clear 
            name_cluster_list = name_cluster.tolist()
            num_cluster_list  = num_cluster.tolist()

            name_cluster_repeat_list = []
            for namee, numm in zip(name_cluster_list,num_cluster):
                name = namee.replace('\'', '').strip()
                repeatname = [name]*numm
                name_cluster_repeat_list.extend(repeatname)

            for name in name_cluster_repeat_list:
                name_array = name.split(' ')
                for nameset in name_array:
                    tlst.append(nameset)    
            #count
            name_counter = Counter(tlst)
            name_counter_topN = Counter(name_counter).most_common(topN)
            #fint topN 
            keywordlist = []
            numberlist  = []
            # print(name_counter_topN)
            for name_rank in name_counter_topN:
                keyword = name_rank[0]
                number  = str(name_rank[1])

                keywordlist.append(keyword)
                numberlist.append(number)

            if len(name_counter_topN) < topN:
                for nn in range(topN- len(name_counter_topN)):
                    keywordlist.append(np.nan) 
                    numberlist.append(np.nan)
            else:
                pass

            temp_df = pd.DataFrame({
                f'clusterName_{j}': keywordlist,
                f'clusterNum_{j}': numberlist,
            })

            df_topN = pd.concat([df_topN,temp_df],axis=1)
        df_topN.to_csv(f'Repeat_{cluster_num}-{topN}/{folder_cate_name}/{folder_cate_name}_Cluster_{topN}.csv',index=False)
    except:
            if not os.path.exists(f'nodata.log'):
                !echo "`date +%Y%m%dT%H%M%S`" > nodata.log
                cmd ='''
                echo %(cate_name)s no data >> nodata.log
                '''%locals()
            else:
                 cmd ='''
                 echo %(cate_name)s no data >> nodata.log
                 '''%locals()                
            os.system(cmd)

/Users/tong/opt/anaconda3/envs/scikit_xgb/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/Users/tong/opt/anaconda3/envs/scikit_xgb/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/Users/tong/opt/anaconda3/envs/scikit_xgb/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/Users/tong/opt/anaconda3/envs/scikit_xgb/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_in